# Local MPChecker

We can use `jorbit` to mimic the MPC's "MPChecker" service. This is a service that, when given a sky coordinate and time, will return a list of all known objects that are in the field of view of a given telescope. This is useful for identifying observed objects in a given field, or for checking for contamination of another science target by solar system objects.

The `jorbit` version is not guaranteed to successfully flag every object: there are a number of simplifications where a particle could fall through the cracks. However, it offers a chance for a rapid first pass, and the the ability to run all computations locally (after downloading some cached data, which is done automatically using Astropy).

In [6]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

from jorbit.mpchecker import mpchecker

For the fastest but crudest check, we can run:

In [7]:
mpchecker(
    coordinate=SkyCoord(ra=0 * u.deg, dec=0 * u.deg),
    time=Time("2025-01-01"),
    radius=10 * u.arcmin,
    extra_precision=False,
)

Unpacked Name,separation,ra,dec,Packed designation,H,G,Epoch,M,Peri,Node,Incl.,e,n,a,U,Reference,#Obs,#Opp,Arc,rms,Coarse Perts,Precise Perts,Computer,Flags,last obs
,arcsec,deg,deg,,,,,,,,,,,,,,,,,,,,,,
str19,float64,float64,float64,str7,str5,str4,str5,str9,str9,str9,str8,str9,str10,str9,str1,str9,str3,str2,str9,str4,str3,str3,str8,str4,str8
2016 BK84,270.4278119801477,359.9281601096068,-0.021951538035608578,K16B84K,17.32,0.15,K24AH,219.42948,140.85942,15.97230,10.97648,0.1729035,0.17932373,3.1144226,0,E2024-TN4,84,7,2010-2024,0.42,M-v,3Ek,MPCLINUX,0000,20241010
(243996) 2001 RP149,299.5286772177697,0.03438721457692705,-0.07576385186187556,O3996,17.80,0.15,K24AH,331.42087,190.52483,205.57069,1.35356,0.0985994,0.30013710,2.2093080,0,E2024-W01,396,15,2000-2024,0.89,M-v,3Ek,MPCLINUX,0000,20241109
(328594) 2009 SU69,328.032332410219,0.040290737741438314,0.08172838417960725,W8594,16.75,0.15,K24AH,139.12830,23.66358,199.35336,5.64561,0.0343690,0.19686043,2.9266027,0,E2024-TI9,223,13,1994-2024,0.99,M-v,3Ek,MPCLINUX,0000,20241006
(192469) 1998 FF48,362.62565453097756,0.04489259766205314,-0.09017237927678272,J2469,15.66,0.15,K24AH,159.54241,2.44130,199.19012,8.44492,0.1114181,0.24065368,2.5598099,0,E2025-B02,877,18,1994-2025,0.78,M-v,3Ek,MPCLINUX,0000,20250116
2020 SE100,365.9350844283869,0.005957379883840604,-0.1014739107881346,K20SA0E,18.59,0.15,K24AH,350.70758,170.43892,203.83276,2.42085,0.1629633,0.23284725,2.6167082,0,E2024-TA7,49,4,2003-2024,0.38,M-v,3Ek,MPCLINUX,0000,20241004
(572408) 2008 GD168,394.4635076095755,359.890512021332,-0.004320642764797974,v2408,14.52,0.15,K24AH,66.34919,274.73593,10.43460,5.35977,0.1149327,0.08318784,5.1971207,0,E2024-U66,109,9,2008-2024,0.98,M-v,3Ek,MPCLINUX,0009,20241023
(167818) 2005 CH8,479.0622857428725,0.1328400530768678,-0.007868018793497944,G7818,16.15,0.15,K24AH,109.74498,219.05555,21.24657,2.00256,0.1478632,0.22879081,2.6475469,0,E2025-B02,650,19,1971-2025,0.86,M-v,3Ek,MPCLINUX,0000,20250116
(123177) 2000 TD66,500.0600640883788,0.026774134766224145,-0.1363007900551778,C3177,16.31,0.15,K24AH,291.52740,53.76172,23.80656,2.18909,0.0453800,0.20291467,2.8680965,0,E2025-B02,395,17,1996-2025,0.93,M-v,3Ek,MPCLINUX,0000,20250116


This should take less than a second (if the appropriate files are downloaded), but makes a few simplifications. First, the true underlying positions of the objects here were integrated using only Newtonian gravity and perturbations from the planets and major asteroids. By neglecting effects like GR, we introduce a ~arsec level error (usually much smaller, but we'll be conservative). Second, when converting these positions into sky coordinates, we've assumed that the observer is located at the geocenter. Depending on the observer's true location, this can introduce $>$arcsec level errors, especially if the object is nearby and the observer is actually a spacecraft like TESS or JWST.

So, for a more accurate check, we can run:

In [8]:
mpchecker(
    coordinate=SkyCoord(ra=0 * u.deg, dec=0 * u.deg),
    time=Time("2025-01-01"),
    radius=10 * u.arcmin,
    extra_precision=True,
    observer="palomar",
)

Unpacked Name,separation,ra,dec,est. Vmag,Packed designation,H,G,Epoch,M,Peri,Node,Incl.,e,n,a,U,Reference,#Obs,#Opp,Arc,rms,Coarse Perts,Precise Perts,Computer,Flags,last obs
,arcsec,deg,deg,mag,,,,,,,,,,,,,,,,,,,,,,
str19,float64,float64,float64,float64[1],str7,str5,str4,str5,str9,str9,str9,str8,str9,str10,str9,str1,str9,str3,str2,str9,str4,str3,str3,str8,str4,str8
2016 BK84,270.28432021529846,359.9283187735462,-0.02233058322974991,23.638584089643366,K16B84K,17.32,0.15,K24AH,219.42948,140.85942,15.97230,10.97648,0.1729035,0.17932373,3.1144226,0,E2024-TN4,84,7,2010-2024,0.42,M-v,3Ek,MPCLINUX,0000,20241010
(243996) 2001 RP149,302.2469225849364,0.03467410789978972,-0.07646283509311654,21.977731705004743,O3996,17.80,0.15,K24AH,331.42087,190.52483,205.57069,1.35356,0.0985994,0.30013710,2.2093080,0,E2024-W01,396,15,2000-2024,0.89,M-v,3Ek,MPCLINUX,0000,20241109
(328594) 2009 SU69,326.9088059954274,0.04047437534991811,0.08128910877633423,22.51069628882137,W8594,16.75,0.15,K24AH,139.12830,23.66358,199.35336,5.64561,0.0343690,0.19686043,2.9266027,0,E2024-TI9,223,13,1994-2024,0.99,M-v,3Ek,MPCLINUX,0000,20241006
(192469) 1998 FF48,364.4511717399132,0.045088747965715485,-0.09064117495488044,21.19595479190651,J2469,15.66,0.15,K24AH,159.54241,2.44130,199.19012,8.44492,0.1114181,0.24065368,2.5598099,0,E2025-B02,877,18,1994-2025,0.78,M-v,3Ek,MPCLINUX,0000,20250116
2020 SE100,368.23783387939346,0.006209053386650075,-0.10209966401928419,23.140906581253827,K20SA0E,18.59,0.15,K24AH,350.70758,170.43892,203.83276,2.42085,0.1629633,0.23284725,2.6167082,0,E2024-TA7,49,4,2003-2024,0.38,M-v,3Ek,MPCLINUX,0000,20241004
(572408) 2008 GD168,394.11007958256664,359.8906207710635,-0.004578729942004327,22.300205386939137,v2408,14.52,0.15,K24AH,66.34919,274.73593,10.43460,5.35977,0.1149327,0.08318784,5.1971207,0,E2024-U66,109,9,2008-2024,0.98,M-v,3Ek,MPCLINUX,0009,20241023
(167818) 2005 CH8,479.8492711750218,0.13303118993162397,-0.00832568691406814,21.779069217340442,G7818,16.15,0.15,K24AH,109.74498,219.05555,21.24657,2.00256,0.1478632,0.22879081,2.6475469,0,E2025-B02,650,19,1971-2025,0.86,M-v,3Ek,MPCLINUX,0000,20250116
(123177) 2000 TD66,501.8946693063552,0.02697180679874228,-0.13678127461271533,21.777211614805058,C3177,16.31,0.15,K24AH,291.52740,53.76172,23.80656,2.18909,0.0453800,0.20291467,2.8680965,0,E2025-B02,395,17,1996-2025,0.93,M-v,3Ek,MPCLINUX,0000,20250116


This will take longer, but should remedy the above issues. Under the hood, this first runs the approximate version to flag which objects were in the field of view, then runs a fresh N-body integration for each on that fully accounts for the observer's location and relativistic effects. The final coordinates should agree to ~1 mas with horizons for times 2010-2030. 

As a plus, we now also can add an extra column, "est. Vmag": since we now know the true distance to the object and its phase angle, we might as well convert its absolute magnitude into an estimated V-band magnitude.

Note though that if any of our above assumptions were egregiously wrong and a particle wasn't flagged, we'll still be out of luck. Also note that this now requires a connection to the internet to request the locations of the observer from Horizons (though the initial positions of the objects are still cached locally).